# Notebook: Análise Exploratória e Preparação de Dados

Este notebook contém o código para análise, tratamento e preparação de dados, incluindo funcionalidades úteis para análise exploratória e modelagem.

# 1) Imports e Configurações

In [1]:
import pandas as pd
import numpy as np
from typing import Dict, List, Tuple, Optional
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import LabelEncoder
import streamlit as st
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import logging

# 2) Funções Utilitárias

In [2]:
# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class DataLoader:
    """
    A class to handle data loading and initial preprocessing operations.
    Includes error handling and validation checks.
    """
    @staticmethod
    def load_dataset(
        file_path: str | Path,
        sep: str = ';',
        encoding: str = 'utf-8'
    ) -> pd.DataFrame:
        """
        Enhanced data loading with validation and error handling.
        
        Args:
            file_path: Path to the data file
            sep: Column separator in the file
            encoding: File encoding
        
        Returns:
            Loaded and validated DataFrame
        
        Raises:
            FileNotFoundError: If the file doesn't exist
            ValueError: If the file is empty or corrupted
        """
        try:
            file_path = Path(file_path)
            if not file_path.exists():
                raise FileNotFoundError(f"File not found: {file_path}")
            
            df = pd.read_csv(file_path, sep=sep, encoding=encoding)
            
            if df.empty:
                raise ValueError("The loaded file is empty")
            
            logger.info(f"Successfully loaded data from {file_path} with shape {df.shape}")
            return df
            
        except Exception as e:
            logger.error(f"Error loading data: {str(e)}")
            raise


In [3]:
# cortesia do prof. William
def limpar_df(df):
    _df = df.dropna(subset=df.columns.difference(['NOME', 'ANO']), how='all') # executa o dropna para todas as colunas sem visualizar a coluna NOME
    _df = _df[~_df.isna().all(axis=1)] # remove linhas com apenas NaN, se tiver algum dado na linha não remove
    return _df

class DataPreprocessor:
    """
    Handles all data preprocessing steps including filtering, cleaning,
    and type conversions with comprehensive validation.
    """
    def __init__(self, indicator_columns: List[str]):
        self.indicator_columns = indicator_columns
        self.label_encoder = LabelEncoder()
    
    def filter_by_year(
        self,
        df: pd.DataFrame,
        year: int,
        name_col: str = 'NOME'
    ) -> pd.DataFrame:
        """
        Filters DataFrame columns for a specific year with improved column handling.
        
        Args:
            df: Input DataFrame
            year: Target year for filtering
            name_col: Name of the identifier column
        
        Returns:
            Filtered DataFrame with standardized column names
        """
        # Get year-specific columns plus identifier
        year_cols = [col for col in df.columns if f'_{year}' in col]
        if not year_cols:
            raise ValueError(f"No columns found for year {year}")
        
        selected_cols = [name_col] + year_cols
        filtered_df = df[selected_cols].copy()
        
        # Standardize column names by removing year suffix
        filtered_df.columns = [col.replace(f'_{year}', '') for col in filtered_df.columns]
        filtered_df['ANO'] = year
        
        return filtered_df

    def clean_data(
        self,
        df: pd.DataFrame
    ) -> pd.DataFrame:
        """
        Args:
            df: Input DataFrame
            threshold: Maximum allowed missing value ratio per row
        
        Returns:
            Cleaned DataFrame
        """
        initial_shape = df.shape
        
        # Remove rows 
        df_cleaned = limpar_df(df)
        
        # Log cleaning results
        removed_rows = initial_shape[0] - df_cleaned.shape[0]
        logger.info(f"Removed {removed_rows} rows ({removed_rows/initial_shape[0]:.2%} of data)")
        
        return df_cleaned

    def convert_numeric(
            self,
            df: pd.DataFrame,
            columns_numeric: List[str]
    ) -> pd.DataFrame:
        
        # Convert indicators to numeric with validation        
        for col in columns_numeric:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')

        return df



In [4]:

class DataVisualizer:
    """
    Handles all visualization tasks with enhanced customization options
    and interactive features.
    """
    @staticmethod
    def create_missing_values_heatmap(
        df: pd.DataFrame,
        title: str,
        width: int = 900,
        height: int = 500
    ) -> go.Figure:
        """
        Creates an interactive heatmap of missing values using Plotly.
        
        Args:
            df: Input DataFrame
            title: Plot title
            width: Plot width
            height: Plot height
        
        Returns:
            Plotly figure object
        """
        missing_matrix = df.isnull().astype(int)
        
        fig = go.Figure(data=go.Heatmap(
            z=missing_matrix,
            x=df.columns,
            y=np.arange(len(df)),
            colorscale=[[0, 'white'], [1, 'red']],
            showscale=True,
            name='Missing Values'
        ))
        
        fig.update_layout(
            title=title,
            width=width,
            height=height,
            xaxis_title="Variables",
            yaxis_title="Observations",
            template="plotly_white"
        )
        
        return fig

    @staticmethod
    def create_indicator_distribution_plot(
        df: pd.DataFrame,
        indicator: str,
        reference_value: float,
        year: int
    ) -> go.Figure:
        """
        Creates an enhanced box plot with violin plot overlay for indicator distribution.
        
        Args:
            df: Input DataFrame
            indicator: Indicator column name
            reference_value: Reference line value
            year: Year for the plot
        
        Returns:
            Plotly figure object
        """
        fig = go.Figure()
        
        # Add violin plot
        fig.add_trace(go.Violin(
            y=df[indicator],
            box_visible=True,
            line_color='blue',
            meanline_visible=True,
            fillcolor='lightblue',
            opacity=0.6,
            name=indicator
        ))
        
        # Add reference line
        fig.add_hline(
            y=reference_value,
            line_dash="dash",
            line_color="red",
            annotation_text=f'Reference: {reference_value:.2f}'
        )
        
        fig.update_layout(
            title=f'{indicator} Distribution ({year})',
            yaxis_title=indicator,
            template="plotly_white",
            showlegend=False
        )
        
        return fig



In [5]:
def process_multi_year_analysis(
    df: pd.DataFrame,
    years: List[int],
    indicator_columns: List[str],
    reference_values: Dict[int, float]
) -> Tuple[Dict[int, pd.DataFrame], List[go.Figure]]:
    """
    Processes and analyzes data for multiple years with comprehensive reporting.
    
    Args:
        df: Input DataFrame
        years: List of years to analyze
        indicator_columns: List of indicator column names
        reference_values: Dictionary of reference values by year
    
    Returns:
        Tuple of processed DataFrames by year and visualization figures
    """
    preprocessor = DataPreprocessor(indicator_columns)
    visualizer = DataVisualizer()
    processed_dfs = {}
    figures = []
    
    for year in years:
        # Process data for each year
        df_year = preprocessor.filter_by_year(df, year)
        df_year_clean = preprocessor.clean_data(df_year)
        processed_dfs[year] = df_year_clean
        
        # Create visualizations
        missing_plot = visualizer.create_missing_values_heatmap(
            df_year_clean,
            f"Missing Values Pattern - {year}"
        )
        figures.append(missing_plot)
        
        # Create distribution plots for each indicator
        for indicator in indicator_columns:
            if indicator in df_year_clean.columns:
                dist_plot = visualizer.create_indicator_distribution_plot(
                    df_year_clean,
                    indicator,
                    reference_values.get(year, 0),
                    year
                )
                figures.append(dist_plot)
    
    return processed_dfs, figures

# Helper function for Streamlit display
def display_streamlit_analysis(figures: List[go.Figure], cols: int = 2):
    """
    Displays analysis results in Streamlit with customizable layout.
    
    Args:
        figures: List of Plotly figures
        cols: Number of columns in the layout
    """
    for i in range(0, len(figures), cols):
        columns = st.columns(cols)
        for j, column in enumerate(columns):
            if i + j < len(figures):
                with column:
                    st.plotly_chart(figures[i + j], use_container_width=True)


# 3) Analise investigatória

In [6]:

# 1) Imports e configurações (já feitos)

# 2) Carregamento da base original

file_path = '../app/data/PEDE_PASSOS_DATASET_FIAP.csv'
data_loader = DataLoader()
df = data_loader.load_dataset(file_path)

2025-02-10 14:07:49,306 - INFO - Successfully loaded data from ..\app\data\PEDE_PASSOS_DATASET_FIAP.csv with shape (1349, 69)


In [7]:
df.columns

Index(['INSTITUICAO_ENSINO_ALUNO_2020', 'NOME', 'IDADE_ALUNO_2020',
       'ANOS_PM_2020', 'FASE_TURMA_2020', 'PONTO_VIRADA_2020', 'INDE_2020',
       'INDE_CONCEITO_2020', 'PEDRA_2020', 'DESTAQUE_IEG_2020',
       'DESTAQUE_IDA_2020', 'DESTAQUE_IPV_2020', 'IAA_2020', 'IEG_2020',
       'IPS_2020', 'IDA_2020', 'IPP_2020', 'IPV_2020', 'IAN_2020', 'FASE_2021',
       'TURMA_2021', 'INSTITUICAO_ENSINO_ALUNO_2021',
       'SINALIZADOR_INGRESSANTE_2021', 'PEDRA_2021', 'INDE_2021', 'IAA_2021',
       'IEG_2021', 'IPS_2021', 'IDA_2021', 'IPP_2021', 'REC_EQUIPE_1_2021',
       'REC_EQUIPE_2_2021', 'REC_EQUIPE_3_2021', 'REC_EQUIPE_4_2021',
       'PONTO_VIRADA_2021', 'IPV_2021', 'IAN_2021', 'NIVEL_IDEAL_2021',
       'DEFASAGEM_2021', 'FASE_2022', 'TURMA_2022', 'ANO_INGRESSO_2022',
       'BOLSISTA_2022', 'INDE_2022', 'CG_2022', 'CF_2022', 'CT_2022',
       'PEDRA_2022', 'DESTAQUE_IEG_2022', 'DESTAQUE_IDA_2022',
       'DESTAQUE_IPV_2022', 'IAA_2022', 'IEG_2022', 'IPS_2022', 'IDA_2022',
       

In [8]:
# Identificar colunas com dados não numéricos
non_numeric_columns = df.select_dtypes(exclude='number').columns
print("\nColunas não numéricas:", non_numeric_columns.tolist())

# Verificar valores nulos
print("Dados nulos por coluna:")
df.isnull().sum()


Colunas não numéricas: ['INSTITUICAO_ENSINO_ALUNO_2020', 'NOME', 'IDADE_ALUNO_2020', 'ANOS_PM_2020', 'FASE_TURMA_2020', 'PONTO_VIRADA_2020', 'INDE_2020', 'INDE_CONCEITO_2020', 'PEDRA_2020', 'DESTAQUE_IEG_2020', 'DESTAQUE_IDA_2020', 'DESTAQUE_IPV_2020', 'IAA_2020', 'IEG_2020', 'IPS_2020', 'IDA_2020', 'IPP_2020', 'IPV_2020', 'IAN_2020', 'TURMA_2021', 'INSTITUICAO_ENSINO_ALUNO_2021', 'SINALIZADOR_INGRESSANTE_2021', 'PEDRA_2021', 'INDE_2021', 'REC_EQUIPE_1_2021', 'REC_EQUIPE_2_2021', 'REC_EQUIPE_3_2021', 'REC_EQUIPE_4_2021', 'PONTO_VIRADA_2021', 'NIVEL_IDEAL_2021', 'TURMA_2022', 'BOLSISTA_2022', 'PEDRA_2022', 'DESTAQUE_IEG_2022', 'DESTAQUE_IDA_2022', 'DESTAQUE_IPV_2022', 'REC_AVA_1_2022', 'REC_AVA_2_2022', 'REC_AVA_3_2022', 'REC_AVA_4_2022', 'INDICADO_BOLSA_2022', 'PONTO_VIRADA_2022', 'NIVEL_IDEAL_2022']
Dados nulos por coluna:


INSTITUICAO_ENSINO_ALUNO_2020    621
NOME                               0
IDADE_ALUNO_2020                 621
ANOS_PM_2020                     621
FASE_TURMA_2020                  621
                                ... 
INDICADO_BOLSA_2022              487
PONTO_VIRADA_2022                487
IPV_2022                         487
IAN_2022                         487
NIVEL_IDEAL_2022                 487
Length: 69, dtype: int64

In [9]:
df = limpar_df(df)

In [10]:
plt.figure(figsize=(20,20))
sns.heatmap(df.isnull(), cbar=False, cmap='viridis')
plt.show()

C:\Users\estevanl\AppData\Local\Temp\ipykernel_25256\4041052429.py:3: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [11]:
# Limpando
colunas_indicadores = ['INDE', 'IAA', 'IEG', 'IPS', 'IDA', 'IPP', 'IPV', 'IAN']

# df['INDE_2020'] = df['INDE_2020'].replace('#NULO!', np.nan)
# df['INDE_2021'] = df['INDE_2021'].replace('#NULO!', np.nan)
# df['INDE_2022'] = df['INDE_2022'].replace('#NULO!', np.nan)

# separando por ano                              
processor = DataPreprocessor(df.columns)
df_2020 = processor.filter_by_year(df, 2020, "NOME")
df_2021 = processor.filter_by_year(df, 2021, "NOME")
df_2022 = processor.filter_by_year(df, 2022, "NOME")


df_2020_cleared = processor.clean_data(df_2020)
df_2021_cleared = processor.clean_data(df_2021)
df_2022_cleared = processor.clean_data(df_2022)

print(f"dados 2020 : {df_2020.shape}")
print(f"dados 2020 limpos:  {df_2020_cleared.shape}")
print(f"dados 2021:  {df_2021.shape}")
print(f"dados 2021 limpos:  {df_2021_cleared.shape}")
print(f"dados 2022 : {df_2022.shape}")
print(f"dados 2022 limpos: {df_2022_cleared.shape}")

2025-02-10 14:07:49,966 - INFO - Removed 621 rows (46.03% of data)
2025-02-10 14:07:49,972 - INFO - Removed 663 rows (49.15% of data)
2025-02-10 14:07:49,977 - INFO - Removed 487 rows (36.10% of data)


dados 2020 : (1349, 20)
dados 2020 limpos:  (728, 20)
dados 2021:  (1349, 22)
dados 2021 limpos:  (686, 22)
dados 2022 : (1349, 32)
dados 2022 limpos: (862, 32)


In [12]:
df_2020_cleared.head(3)

,NOME,INSTITUICAO_ENSINO_ALUNO,IDADE_ALUNO,ANOS_PM,FASE_TURMA,PONTO_VIRADA,INDE,INDE_CONCEITO,PEDRA,DESTAQUE_IEG,DESTAQUE_IDA,DESTAQUE_IPV,IAA,IEG,IPS,IDA,IPP,IPV,IAN,ANO
0,ALUNO-1,Escola Pública,11,2,2H,Não,7.883752,B,Ametista,Seu destaque em 2020: A sua boa entrega das li...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Seu destaque em 2020: A sua boa integração aos...,8.500020000000001,8.7,7.5,7,5.9375,7.75,10,2020
2,ALUNO-3,Escola Pública,12,2,3H,Não,7.856389666666666,B,Ametista,Seu destaque em 2020: A sua boa entrega das li...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Seu destaque em 2020: A sua boa integração aos...,7.91667,8.9,7.5,5.5,8.125,8.111113333333334,10,2020
3,ALUNO-4,Escola Pública,10,2,1D,Não,5.076252,D,Quartzo,Ponto a melhorar em 2021: Melhorar a sua entre...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Ponto a melhorar em 2021: Integrar-se mais aos...,8.000020000000001,4.1,6.875,0,7.1875,7.75,5,2020


In [13]:
df_2020_cleared = processor.convert_numeric(df_2020_cleared, columns_numeric=colunas_indicadores)
df_2021_cleared = processor.convert_numeric(df_2021_cleared, columns_numeric=colunas_indicadores)
df_2022_cleared = processor.convert_numeric(df_2022_cleared, columns_numeric=colunas_indicadores)
df_2020_cleared.head(3)

print(f"dados 2020 limpos:  {df_2020_cleared.shape}")
print(f"dados 2021 limpos:  {df_2021_cleared.shape}")
print(f"dados 2022 limpos: {df_2022_cleared.shape}")

dados 2020 limpos:  (728, 20)
dados 2021 limpos:  (686, 22)
dados 2022 limpos: (862, 32)


In [14]:
# Identificar colunas com dados não numéricos
non_numeric_columns = df_2020_cleared.select_dtypes(exclude='number').columns
print("\nColunas não numéricas: df_2020_cleared\n" , non_numeric_columns.tolist())

# Identificar colunas com dados não numéricos
non_numeric_columns = df_2021_cleared.select_dtypes(exclude='number').columns
print("\nColunas não numéricas: df_2021_cleared\n", non_numeric_columns.tolist())

# Identificar colunas com dados não numéricos
non_numeric_columns = df_2022_cleared.select_dtypes(exclude='number').columns
print("\nColunas não numéricas: df_2022_cleared\n", non_numeric_columns.tolist())




Colunas não numéricas: df_2020_cleared
 ['NOME', 'INSTITUICAO_ENSINO_ALUNO', 'IDADE_ALUNO', 'ANOS_PM', 'FASE_TURMA', 'PONTO_VIRADA', 'INDE_CONCEITO', 'PEDRA', 'DESTAQUE_IEG', 'DESTAQUE_IDA', 'DESTAQUE_IPV']

Colunas não numéricas: df_2021_cleared
 ['NOME', 'TURMA', 'INSTITUICAO_ENSINO_ALUNO', 'SINALIZADOR_INGRESSANTE', 'PEDRA', 'REC_EQUIPE_1', 'REC_EQUIPE_2', 'REC_EQUIPE_3', 'REC_EQUIPE_4', 'PONTO_VIRADA', 'NIVEL_IDEAL']

Colunas não numéricas: df_2022_cleared
 ['NOME', 'TURMA', 'BOLSISTA', 'PEDRA', 'DESTAQUE_IEG', 'DESTAQUE_IDA', 'DESTAQUE_IPV', 'REC_AVA_1', 'REC_AVA_2', 'REC_AVA_3', 'REC_AVA_4', 'INDICADO_BOLSA', 'PONTO_VIRADA', 'NIVEL_IDEAL']


In [15]:
def verificar_nao_numericos(df, colunas):
    resultado = {}
    
    for coluna in colunas:
        # Filtrar valores não numéricos
        valores_nao_numericos = df[coluna][~pd.to_numeric(df[coluna], errors='coerce').notna()]
        
        if not valores_nao_numericos.empty:
            # Contar ocorrências de valores não numéricos
            contagem_valores = valores_nao_numericos.value_counts().to_dict()
            
            resultado[coluna] = {
                'quantidade_nao_numericos': len(valores_nao_numericos),
                'valores_nao_numericos': contagem_valores
            }
    
    return resultado

nao_numericos_2020 = verificar_nao_numericos(df_2020_cleared, colunas_indicadores)
print("valores não numericos para 2020 nos indicies: ")
print(nao_numericos_2020)
nao_numericos_2021 = verificar_nao_numericos(df_2021_cleared, colunas_indicadores)
print("valores não numericos para 2021 nos indicies: ")
print(nao_numericos_2021)
nao_numericos_2022 = verificar_nao_numericos(df_2022_cleared, colunas_indicadores)
print("valores não numericos para 2022 nos indicies: ")
print(nao_numericos_2022)

valores não numericos para 2020 nos indicies: 
{'INDE': {'quantidade_nao_numericos': 1, 'valores_nao_numericos': {}}, 'IAA': {'quantidade_nao_numericos': 1, 'valores_nao_numericos': {}}, 'IEG': {'quantidade_nao_numericos': 1, 'valores_nao_numericos': {}}, 'IPS': {'quantidade_nao_numericos': 1, 'valores_nao_numericos': {}}, 'IDA': {'quantidade_nao_numericos': 1, 'valores_nao_numericos': {}}, 'IPP': {'quantidade_nao_numericos': 1, 'valores_nao_numericos': {}}, 'IPV': {'quantidade_nao_numericos': 1, 'valores_nao_numericos': {}}, 'IAN': {'quantidade_nao_numericos': 1, 'valores_nao_numericos': {}}}
valores não numericos para 2021 nos indicies: 
{'INDE': {'quantidade_nao_numericos': 2, 'valores_nao_numericos': {}}}
valores não numericos para 2022 nos indicies: 
{}


In [16]:
# Configurar o layout dos heatmaps
fig, axes = plt.subplots(2, 2, figsize=(15, 15))  # 1 linha, 3 colunas

sns.heatmap(df_2020_cleared.isnull(), cbar=False, cmap='viridis', ax=axes.flat[0])
axes.flat[0].set_title("dados ano 2020")

sns.heatmap(df_2021_cleared.isnull(), cbar=False, cmap='viridis',ax=axes.flat[1])
axes.flat[1].set_title("dados ano 2021")

sns.heatmap(df_2022_cleared.isnull(), cbar=False, cmap='viridis', ax=axes.flat[2])
axes.flat[2].set_title("dados ano 2022")

# Ajustar espaçamento
plt.tight_layout()
plt.show()

C:\Users\estevanl\AppData\Local\Temp\ipykernel_25256\2022686780.py:15: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [17]:
# verificando por nulos em todos os indicadores
print(df_2020_cleared[['INDE', 'IAA', 'IEG', 'IPS', 'IDA', 'IPP', 'IPV', 'IAN']].isna().sum())

df_2020_cleared.drop(index=df_2020_cleared[df_2020_cleared['INDE'].isna()].index, inplace=True)
df_2020_cleared.head(2)

INDE    1
IAA     1
IEG     1
IPS     1
IDA     1
IPP     1
IPV     1
IAN     1
dtype: int64


,NOME,INSTITUICAO_ENSINO_ALUNO,IDADE_ALUNO,ANOS_PM,FASE_TURMA,PONTO_VIRADA,INDE,INDE_CONCEITO,PEDRA,DESTAQUE_IEG,DESTAQUE_IDA,DESTAQUE_IPV,IAA,IEG,IPS,IDA,IPP,IPV,IAN,ANO
0,ALUNO-1,Escola Pública,11,2,2H,Não,7.883752,B,Ametista,Seu destaque em 2020: A sua boa entrega das li...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Seu destaque em 2020: A sua boa integração aos...,8.50002,8.7,7.5,7.0,5.9375,7.750000,10.0,2020
2,ALUNO-3,Escola Pública,12,2,3H,Não,7.856390,B,Ametista,Seu destaque em 2020: A sua boa entrega das li...,Ponto a melhorar em 2021: Empenhar-se mais nas...,Seu destaque em 2020: A sua boa integração aos...,7.91667,8.9,7.5,5.5,8.1250,8.111113,10.0,2020


In [18]:
print(df_2021_cleared[['INDE', 'IAA', 'IEG', 'IPS', 'IDA', 'IPP', 'IPV', 'IAN']].isna().sum())

df_2021_cleared.drop(index=df_2021_cleared[df_2021_cleared['INDE'].isna()].index, inplace=True)
df_2021_cleared.head(2)

INDE    2
IAA     0
IEG     0
IPS     0
IDA     0
IPP     0
IPV     0
IAN     0
dtype: int64


,NOME,FASE,TURMA,INSTITUICAO_ENSINO_ALUNO,SINALIZADOR_INGRESSANTE,PEDRA,INDE,IAA,IEG,IPS,...,REC_EQUIPE_1,REC_EQUIPE_2,REC_EQUIPE_3,REC_EQUIPE_4,PONTO_VIRADA,IPV,IAN,NIVEL_IDEAL,DEFASAGEM,ANO
0,ALUNO-1,2.0,G,Rede Decisão,Ingressante,Ametista,7.319,7.5,6.3,6.9,...,Promovido de Fase,Promovido de Fase,Mantido na Fase atual,Não avaliado,Não,8.1,5.0,Nível 3 (7o e 8o ano),-1.0,2021
2,ALUNO-3,3.0,F,Escola Pública,Veterano,Ametista,7.021,7.1,6.9,6.3,...,Promovido de Fase,Promovido de Fase,Mantido na Fase atual,Não avaliado,Não,7.2,10.0,Nível 3 (7o e 8o ano),0.0,2021


In [19]:
print(df_2022_cleared[['INDE', 'IAA', 'IEG', 'IPS', 'IDA', 'IPP', 'IPV', 'IAN']].isna().sum())

INDE    0
IAA     0
IEG     0
IPS     0
IDA     0
IPP     0
IPV     0
IAN     0
dtype: int64


In [20]:
print(f"dados 2020 : {df_2020.shape}")
print(f"dados 2020 limpos:  {df_2020_cleared.shape}")
print(f"dados 2021:  {df_2021.shape}")
print(f"dados 2021 limpos:  {df_2021_cleared.shape}")
print(f"dados 2022 : {df_2022.shape}")
print(f"dados 2022 limpos: {df_2022_cleared.shape}")

dados 2020 : (1349, 20)
dados 2020 limpos:  (727, 20)
dados 2021:  (1349, 22)
dados 2021 limpos:  (684, 22)
dados 2022 : (1349, 32)
dados 2022 limpos: (862, 32)


In [21]:
print(df_2021_cleared.columns)
print(df_2020_cleared.columns)
print(df_2022_cleared.columns)


Index(['NOME', 'FASE', 'TURMA', 'INSTITUICAO_ENSINO_ALUNO',
       'SINALIZADOR_INGRESSANTE', 'PEDRA', 'INDE', 'IAA', 'IEG', 'IPS', 'IDA',
       'IPP', 'REC_EQUIPE_1', 'REC_EQUIPE_2', 'REC_EQUIPE_3', 'REC_EQUIPE_4',
       'PONTO_VIRADA', 'IPV', 'IAN', 'NIVEL_IDEAL', 'DEFASAGEM', 'ANO'],
      dtype='object')
Index(['NOME', 'INSTITUICAO_ENSINO_ALUNO', 'IDADE_ALUNO', 'ANOS_PM',
       'FASE_TURMA', 'PONTO_VIRADA', 'INDE', 'INDE_CONCEITO', 'PEDRA',
       'DESTAQUE_IEG', 'DESTAQUE_IDA', 'DESTAQUE_IPV', 'IAA', 'IEG', 'IPS',
       'IDA', 'IPP', 'IPV', 'IAN', 'ANO'],
      dtype='object')
Index(['NOME', 'FASE', 'TURMA', 'ANO_INGRESSO', 'BOLSISTA', 'INDE', 'CG', 'CF',
       'CT', 'PEDRA', 'DESTAQUE_IEG', 'DESTAQUE_IDA', 'DESTAQUE_IPV', 'IAA',
       'IEG', 'IPS', 'IDA', 'NOTA_PORT', 'NOTA_MAT', 'NOTA_ING', 'QTD_AVAL',
       'IPP', 'REC_AVA_1', 'REC_AVA_2', 'REC_AVA_3', 'REC_AVA_4',
       'INDICADO_BOLSA', 'PONTO_VIRADA', 'IPV', 'IAN', 'NIVEL_IDEAL', 'ANO'],
      dtype='object')


# Fase e turma de 2020

In [22]:
# fazer somente uma vez 
df_2020_cleared['FASE'] = df_2020_cleared['FASE_TURMA'].str[0]
df_2020_cleared['TURMA'] = df_2020_cleared['FASE_TURMA'].str[1]
df_2020_cleared['FASE'] = df_2020_cleared['FASE'].replace('D','2')
df_2020_cleared['TURMA'] = df_2020_cleared['TURMA'].replace('2','D')
df_2020_cleared.drop('FASE_TURMA', axis=1, inplace=True)

print(df_2020_cleared.columns)


Index(['NOME', 'INSTITUICAO_ENSINO_ALUNO', 'IDADE_ALUNO', 'ANOS_PM',
       'PONTO_VIRADA', 'INDE', 'INDE_CONCEITO', 'PEDRA', 'DESTAQUE_IEG',
       'DESTAQUE_IDA', 'DESTAQUE_IPV', 'IAA', 'IEG', 'IPS', 'IDA', 'IPP',
       'IPV', 'IAN', 'ANO', 'FASE', 'TURMA'],
      dtype='object')


In [23]:

df_2020_cleared ['FASE'] = pd.to_numeric(df_2020_cleared ['FASE'], errors='coerce')

In [24]:
print(df.shape)
print(df_2020_cleared.shape)
print(df_2020_cleared.describe())
df_2020_cleared.head(5)

(1349, 69)
(727, 21)
             INDE         IAA         IEG         IPS         IDA         IPP  \
count  727.000000  727.000000  727.000000  727.000000  727.000000  727.000000   
mean     7.296111    8.368882    7.681155    6.737448    6.322260    7.067859   
std      1.199925    1.727980    2.382122    1.374011    2.960073    1.988054   
min      3.065002    0.000000    0.000000    2.500000    0.000000    0.000000   
25%      6.569584    7.958350    6.600000    5.625000    4.666667    6.953125   
50%      7.580834    8.750010    8.500000    7.500000    7.000000    7.500000   
75%      8.178960    9.166680    9.500000    7.500000    8.666667    8.125000   
max      9.712504   10.000020   10.000000   10.000000   10.000000   10.000000   

              IPV         IAN     ANO        FASE  
count  727.000000  727.000000   727.0  727.000000  
mean     7.242243    7.431224  2020.0    2.669876  
std      1.775110    2.556944     0.0    2.089845  
min      0.000000    2.500000  2020.0    

,NOME,INSTITUICAO_ENSINO_ALUNO,IDADE_ALUNO,ANOS_PM,PONTO_VIRADA,INDE,INDE_CONCEITO,PEDRA,DESTAQUE_IEG,DESTAQUE_IDA,...,IAA,IEG,IPS,IDA,IPP,IPV,IAN,ANO,FASE,TURMA
0,ALUNO-1,Escola Pública,11,2,Não,7.883752,B,Ametista,Seu destaque em 2020: A sua boa entrega das li...,Ponto a melhorar em 2021: Empenhar-se mais nas...,...,8.50002,8.7,7.500,7.000000,5.9375,7.750000,10.0,2020,2,H
2,ALUNO-3,Escola Pública,12,2,Não,7.856390,B,Ametista,Seu destaque em 2020: A sua boa entrega das li...,Ponto a melhorar em 2021: Empenhar-se mais nas...,...,7.91667,8.9,7.500,5.500000,8.1250,8.111113,10.0,2020,3,H
3,ALUNO-4,Escola Pública,10,2,Não,5.076252,D,Quartzo,Ponto a melhorar em 2021: Melhorar a sua entre...,Ponto a melhorar em 2021: Empenhar-se mais nas...,...,8.00002,4.1,6.875,0.000000,7.1875,7.750000,5.0,2020,1,D
4,ALUNO-5,Escola Pública,10,1,Não,8.077085,B,Ametista,Seu destaque em 2020: A sua boa entrega das li...,Seu destaque em 2020: As suas boas notas na Pa...,...,7.50002,8.0,7.500,7.500000,8.4375,8.166665,10.0,2020,2,M
7,ALUNO-8,Rede Decisão/União,14,3,Sim,8.381391,B,Ametista,Seu destaque em 2020: A sua boa entrega das li...,Ponto a melhorar em 2021: Empenhar-se mais nas...,...,8.33334,9.9,4.375,7.333333,8.7500,8.944450,10.0,2020,4,L


In [25]:
print(df.shape)
print(df_2021_cleared.shape)
print(df_2021_cleared.describe())
df_2021_cleared.head(5)

(1349, 69)
(684, 22)
             FASE        INDE         IAA         IEG         IPS         IDA  \
count  684.000000  684.000000  684.000000  684.000000  684.000000  684.000000   
mean     2.283626    6.888016    8.155702    6.842544    6.848977    5.430556   
std      1.796766    1.354180    2.196459    2.434306    1.455856    2.142955   
min      0.000000    2.468000    0.000000    0.000000    0.000000    0.000000   
25%      1.000000    6.066250    7.900000    5.300000    6.900000    4.100000   
50%      2.000000    7.166000    8.800000    7.200000    7.500000    5.900000   
75%      3.000000    7.892750    9.500000    8.800000    7.500000    6.925000   
max      7.000000    9.294000   10.000000   10.000000    8.800000    9.500000   

              IPP         IPV         IAN   DEFASAGEM     ANO  
count  684.000000  684.000000  684.000000  684.000000   684.0  
mean     7.598538    7.429240    6.900585   -0.751462  2021.0  
std      0.992297    1.579237    2.484723    0.791378    

,NOME,FASE,TURMA,INSTITUICAO_ENSINO_ALUNO,SINALIZADOR_INGRESSANTE,PEDRA,INDE,IAA,IEG,IPS,...,REC_EQUIPE_1,REC_EQUIPE_2,REC_EQUIPE_3,REC_EQUIPE_4,PONTO_VIRADA,IPV,IAN,NIVEL_IDEAL,DEFASAGEM,ANO
0,ALUNO-1,2.0,G,Rede Decisão,Ingressante,Ametista,7.319,7.5,6.3,6.9,...,Promovido de Fase,Promovido de Fase,Mantido na Fase atual,Não avaliado,Não,8.1,5.0,Nível 3 (7o e 8o ano),-1.0,2021
2,ALUNO-3,3.0,F,Escola Pública,Veterano,Ametista,7.021,7.1,6.9,6.3,...,Promovido de Fase,Promovido de Fase,Mantido na Fase atual,Não avaliado,Não,7.2,10.0,Nível 3 (7o e 8o ano),0.0,2021
4,ALUNO-5,2.0,L,Escola Pública,Veterano,Ametista,7.399,7.4,7.5,7.5,...,Promovido de Fase,Mantido na Fase atual,Mantido na Fase atual,Não avaliado,Não,7.4,10.0,Nível 2 (5o e 6o ano),0.0,2021
7,ALUNO-8,5.0,L,Rede Decisão,Veterano,Topázio,8.269,8.8,8.6,7.5,...,Promovido de Fase + Bolsa,Promovido de Fase,Promovido de Fase,Mantido na Fase atual,Não,8.5,10.0,Nível 5 (1o EM),0.0,2021
10,ALUNO-11,1.0,A,Rede Decisão,Veterano,Topázio,8.990,9.0,10.0,6.3,...,Promovido de Fase,Mantido na Fase atual,Mantido na Fase atual,Não avaliado,Sim,9.7,10.0,Nível 1 (4o ano),0.0,2021


In [26]:
print(df.shape)
print(df_2022_cleared.shape)
print(df_2022_cleared.describe())
df_2022_cleared.head(5)

(1349, 69)
(862, 32)
             FASE  ANO_INGRESSO        INDE          CG          CF  \
count  862.000000    862.000000  862.000000  862.000000  862.000000   
mean     2.110209   2020.496520    7.028338  431.500000   75.396752   
std      1.802222      1.787649    1.029912  248.982262   52.314184   
min      0.000000   2016.000000    3.031806    1.000000    1.000000   
25%      1.000000   2019.000000    6.472561  216.250000   30.000000   
50%      2.000000   2021.000000    7.193950  431.500000   66.500000   
75%      3.000000   2022.000000    7.749605  646.750000  118.000000   
max      7.000000   2022.000000    9.441522  862.000000  192.000000   

               CT         IAA         IEG         IPS         IDA   NOTA_PORT  \
count  862.000000  862.000000  862.000000  862.000000  862.000000  860.000000   
mean     6.617169    8.263065    7.880728    6.901102    6.070688    5.796318   
std      3.989485    2.080264    1.649023    1.070202    2.059346    2.422292   
min      1.0000

,NOME,FASE,TURMA,ANO_INGRESSO,BOLSISTA,INDE,CG,CF,CT,PEDRA,...,REC_AVA_1,REC_AVA_2,REC_AVA_3,REC_AVA_4,INDICADO_BOLSA,PONTO_VIRADA,IPV,IAN,NIVEL_IDEAL,ANO
1,ALUNO-2,0.0,K,2022.0,Não,7.675509,245.0,66.0,6.0,Ametista,...,Mantido na Fase + Bolsa,Mantido na Fase atual,NaN,NaN,Sim,Não,7.916665,5.0,Fase 2 (5º e 6º ano),2022
3,ALUNO-4,2.0,J,2022.0,Não,7.117065,451.0,69.0,3.0,Ametista,...,Promovido de Fase,Mantido na Fase atual,Promovido de Fase,NaN,Não,Não,8.055553,5.0,Fase 3 (7º e 8º ano),2022
4,ALUNO-5,2.0,L,2019.0,Não,7.232336,415.0,61.0,12.0,Ametista,...,Mantido na Fase atual,Promovido de Fase,Mantido na Fase atual,NaN,Não,Não,7.833337,5.0,Fase 3 (7º e 8º ano),2022
5,ALUNO-6,2.0,K,2022.0,Não,7.741669,221.0,35.0,2.0,Ametista,...,Mantido na Fase atual,Promovido de Fase,Mantido na Fase atual,NaN,Não,Não,7.500000,10.0,Fase 2 (5º e 6º ano),2022
6,ALUNO-7,1.0,U,2022.0,Não,7.406743,357.0,93.0,7.0,Ametista,...,Promovido de Fase,Promovido de Fase,NaN,NaN,Não,Não,7.916670,5.0,Fase 2 (5º e 6º ano),2022


In [27]:
# total df 20 21 22
df_tot = pd.concat([df_2020_cleared, df_2021_cleared, df_2022_cleared], ignore_index=True)

In [28]:
df_tot.head(5)
sns.heatmap(df_tot.isna(), cbar=False, cmap='viridis')
plt.show()

C:\Users\estevanl\AppData\Local\Temp\ipykernel_25256\2569242173.py:3: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


# Dados limpos para o streamlit 

In [29]:


df_2020_cleared.to_csv('../app/data/pede_2020.csv', sep=';')
df_2021_cleared.to_csv('../app/data/pede_2021.csv', sep=';')
df_2022_cleared.to_csv('../app/data/pede_2022.csv', sep=';')
df_tot.to_csv('../app/data/pede_total.csv', sep=';')